#Searching for global minimum
<font color='grey' size='2' > Created by Parisa Hosseinzadeh for *Protein Engineering and Design*, Winter 2022

NOTE: Please download this notebook and add it to your folder.

In this activity we will be using three algorithms to find solution to our design problem. Our design problem is as follows: 

I am trying to find the best amino acids (including their type and their rotamers) in three positions in my cyclic peptide in order to stabilize its structure. We have 3 possible combinations of (aa, rotamer) for each position, giving us a total of 27 possibilities.

Below is a schematic view of the problem:

<img src='https://drive.google.com/uc?export=view&id=1UiZBxhvAezqFMWcWDxDy2Ib3ZYcWtki4' width='400'>

<font size='1'>Image Courtesy: Vikram Khipple Mulligan



In [1]:
#@title Importing necessary modules
#@markdown Run this cell to download 
#@markdown necessary modules to run the code.

#importing modules necessary for plotting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import time

In [2]:
#@title Loading rotamer energies
#@markdown Run this cell to load 
#@markdown energies of each rotamer 
#@markdown and their interacrtions.

#@markdown <img src='https://drive.google.com/uc?export=view&id=1-anN05ZWYclxG8RBsxi40XVsbuGukCjw' width='250'>

#Individual energies
intra_e = {
    '1.1': 15.2,
    '1.2':3.3,
    '1.3':7.1,
    '2.1':2.1,
    '2.2':6.8,
    '2.3':3.1,
    '3.1':3.1,
    '3.2':3.5,
    '3.3':1.5,
}

#Paired energies
inter_e = {
      '1.1-2.1':5.3,
      '1.1-2.2':1.6, 
      '1.1-2.3':0.7,
      '1.2-2.1':5.8, 
      '1.2-2.2':4.8, 
      '1.2-2.3':2.3,
      '1.3-2.1':3.1, 
      '1.3-2.2':3.5,
      '1.3-2.3':3.7,
      '1.1-3.1':7.9,
      '1.1-3.2':4.3, 
      '1.1-3.3':4.1,
      '1.2-3.1':1.3, 
      '1.2-3.2':5.3, 
      '1.2-3.3':5.8,
      '1.3-3.1':1.4, 
      '1.3-3.2':1.3,
      '1.3-3.3':1.1,
      '2.1-3.1':3.0,
      '2.1-3.2':3.1, 
      '2.1-3.3':2.9,
      '2.2-3.1':3.0, 
      '2.2-3.2':3.6, 
      '2.2-3.3':1.7,
      '2.3-3.1':2.5, 
      '2.3-3.2':4.1,
      '2.3-3.3':0.7,
      '2.1-1.1':5.3,
      '2.2-1.1':1.6, 
      '2.3-1.1':0.7,
      '2.1-1.2':5.8, 
      '2.2-1.2':4.8, 
      '2.3-1.2':2.3,
      '2.1-1.3':3.1, 
      '2.2-1.3':3.5,
      '2.3-1.3':3.7,
      '3.1-1.1':7.9,
      '3.2-1.1':4.3, 
      '3.3-1.1':4.1,
      '3.1-1.2':1.3, 
      '3.2-1.2':5.3, 
      '3.3-1.2':5.8,
      '3.1-1.3':1.4, 
      '3.2-1.3':1.3,
      '3.3-1.3':1.1,
      '3.1-2.1':3.0,
      '3.2-2.1':3.1, 
      '3.3-2.1':2.9,
      '3.1-2.2':3.0, 
      '3.2-2.2':3.6, 
      '3.3-2.2':1.7,
      '3.1-2.3':2.5, 
      '3.2-2.3':4.1,
      '3.3-2.3':0.7,
}


In [ ]:
#@title Exhaustive sampling
#@markdown Conceptually, the simplest method for getting energies
#@markdown is to try all combinations and rank them.

#@markdown Run this cell to get the rank.
#@markdown 1.1 means amino acid 1, rotamer 1.

start = time.time()

def get_combined_val(a1,a2,a3):
  E1 = intra_e[a1] + intra_e[a2] + intra_e[a3]
  E2 = inter_e[a1+'-'+a2] + inter_e[a1+'-'+a3] + inter_e[a2+'-'+a3]
  time.sleep(1)

  return E1+E2

for _ in range(1):

  A = ['1.1','1.2', '1.3']
  B = ['2.1','2.2', '2.3']
  C = ['3.1','3.2', '3.3']

  energy_dict = {}
  for a in A:
    for b in B:
      for c in C:
        energy_dict[a+'-'+b+'-'+c]=get_combined_val(a,b,c)
      
  x_val = list(energy_dict.keys())
  y_val = list(energy_dict.values())

end = time.time()
print('The run time is:', end - start)

plt.xticks(rotation=90)
plt.bar(x=x_val, height=y_val,)




In [ ]:
#@title Branch-and-boubd
#@markdown In this method, we start testing combinations
#@markdown and will remove the ones that don't work early on.
#@markdown This ensures that we don't search the entire space.

#@markdown Run this cell to get the rank.
#@markdown 1.1 means amino acid 1, rotamer 1.

start = time.time()

def get_val(a1,a2):
  E1 = intra_e[a1] + intra_e[a2] 
  E2 = inter_e[a1+'-'+a2]
  time.sleep(0.1)

  return E1+E2

for _ in range(1):

  A = ['1.1','1.2', '1.3']
  B = ['2.1','2.2', '2.3']
  C = ['3.1','3.2', '3.3']

  energy_dict = {}
  threshold = 15
  for a in A:
    for b in B:
      if get_val(a, b) > threshold:
        continue
      for c in C:
        energy_dict[a+'-'+b+'-'+c]=get_combined_val(a,b,c)
      
  x_val = list(energy_dict.keys())
  y_val = list(energy_dict.values())

end = time.time()
print('The run time is:', end - start)

plt.xticks(rotation=90)
plt.bar(x=x_val, height=y_val,)


In [ ]:
#@title Gradient descent
#@markdown In this method, start taking random steps down the path,
#@markdown with some probability of allowing some jumps over hills.
#@markdown This ensures that we don't search the entire space.

#@markdown Run this cell to get the rank.
#@markdown 1.1 means amino acid 1, rotamer 1.

start = time.time()

steps = 2
tries = 1
early_stop = True

A = ['1.1','1.2', '1.3']
B = ['2.1','2.2', '2.3']
C = ['3.1','3.2', '3.3']
pos = [A, B, C]
locs = [random.randint(0,2) for i in range(3)]
seq = [A[locs[0]],B[locs[1]],C[locs[2]]]
E_min = get_combined_val(seq[0],seq[1],seq[2])

for j in range(tries):
  if j!= 0:
    A = ['1.1','1.2', '1.3']
    B = ['2.1','2.2', '2.3']
    C = ['3.1','3.2', '3.3']
    pos = [A, B, C]
    locs = [random.randint(0,2) for i in range(3)]
    seq = [A[locs[0]],B[locs[1]],C[locs[2]]]
    new_E = get_combined_val(seq[0],seq[1],seq[2])
    if new_E < E_min:
      E_min = new_E
  else:
    new_E = E_min

  A = [a for a in A if a != A[locs[0]]]
  B = [b for b in B if b != B[locs[0]]]
  C = [c for c in C if c != C[locs[0]]]

  for i in range(steps):
    counter = 0
    mut_site = random.randint(0,2)
    if len(pos[mut_site]) == 1:
      new_num = 0
    elif len(pos[mut_site]) == 0:
      break
    else:
      new_num = random.randint(0,len(pos[mut_site])-1)
    new_aa = pos[mut_site][new_num]
    #accounting for changes in intra-E
    new_E = new_E + intra_e[
              pos[mut_site][new_num]
            ]-intra_e[seq[mut_site]]
    seq_l = [seq[i] for i in range(
              len(seq)
              ) if i != mut_site]
    #accounting for bi-molecular interactions
    new_E = new_E + inter_e[
                    new_aa+'-'+seq_l[0]
                ] + inter_e[
                    new_aa+'-'+seq_l[1]
                ] - inter_e[
                    seq[mut_site]+'-'+seq_l[0]
                ] - inter_e[
                    seq[mut_site]+'-'+seq_l[1]
                ]
    new_seq =  [a for a in seq]
    # new_E = get_combined_val(
    #     new_seq[0],new_seq[1],new_seq[2])
    # pos[mut_site] = [i for i in pos[mut_site] if i != new_aa]
    if new_E < E_min:
      E_min = new_E
      seq = new_seq
    else:
      if counter > 2:
        break
      r_h = random.random()
      if r_h > 0.99:
        E_min = new_E
        seq = new_seq
    counter +=1

end = time.time()
print('The run time is:', end - start)

print(E_min, seq)



